In [1]:
%load_ext autoreload
%autoreload 2
# %load_ext lab_black

In [2]:
# Filter out hard questions
from collections import defaultdict
import numpy as np
import random
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

/cluster/home/kevidu/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cluster/home/kevidu/venv/lib/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
# IMPORTANT: Run as if from project root so that imports work.
pardir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
os.chdir(pardir)
os.getcwd()

'/cluster/work/cotterell/kdu/context-vs-prior-finetuning'

In [4]:
from preprocessing.dataset import load_dataset_from_path
from preprocessing.utils import format_query, evaluate_model
from model_utils.utils import evaluate_model

In [5]:
SEED = 1
random.seed(SEED)
np.random.seed(SEED)

In [43]:
ROOT_DATA_DIR = "data/Yago/"
RAW_DATA_PATH = os.path.join(ROOT_DATA_DIR, "yago_qec.json")
FILTERED_RAW_DATA_PATH = os.path.join(ROOT_DATA_DIR, "llama2chat_yago_qec.json")
dataset = load_dataset_from_path(RAW_DATA_PATH)
len(dataset.keys())
dataset[:1]

In [7]:
model_id = "unsloth/llama-2-7b-chat-bnb-4bit"
device = "auto"
dtype = "auto"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map=device,
    torch_dtype=dtype,
    # attn_implementation=attn_implementation,
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/cluster/home/kevidu/venv/lib/python3.11/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model.config._name_or_path)
tokenizer.padding_side = "right" 

In [9]:
my_dataset = defaultdict(list)
# num_entities = 50
for query_id, qec in tqdm(dataset.items()):
    ents_and_answers = list(zip(qec["entities"], qec["answers"]))
    random.shuffle(ents_and_answers)
    # ents_and_answers = ents_and_answers[:num_entities]
    for entity, answer in ents_and_answers:
        for qt, qfs in qec["query_forms"].items():
            for qf in qfs:
                if not qf.startswith("Q:"):
                    query = format_query(
                        query=qf, entity=(entity,), context="", answer=answer
                    )
                    my_dataset["text"] += [query]
                    my_dataset["labels"] += [
                        answer if qt == "open" else "No"
                    ]

                    # Add metadata shared between both examples
                    my_dataset["entity"] += [entity]
                    my_dataset["query_id"] += [query_id]
                    my_dataset["query_type"] += [qt]


df_all = pd.DataFrame.from_dict(my_dataset)
df_all

100%|██████████| 125/125 [00:00<00:00, 387.66it/s]


,text,labels,entity,query_id,query_type
0,'Lady Blue' is about,Chicago Police Department,Lady Blue,http://schema.org/about,open
1,'The Guys' is about,New York City Fire Department,The Guys,http://schema.org/about,open
2,'installation of Yang di-Pertuan Agong XVI' is...,Abdullah of Pahang,installation of Yang di-Pertuan Agong XVI,http://schema.org/about,open
3,'Utilities Act 2000' is about,public utility,Utilities Act 2000,http://schema.org/about,open
4,'905' is about,cloning,905,http://schema.org/about,open
...,...,...,...,...,...
109050,Qingshui District is the terminus of,Provincial Highway 10,Qingshui District,reverse-http://yago-knowledge.org/resource/ter...,open
109051,Beppu is the terminus of,Japan National Route 213,Beppu,reverse-http://yago-knowledge.org/resource/ter...,open
109052,San Antonio is the terminus of,U.S. Route 380,San Antonio,reverse-http://yago-knowledge.org/resource/ter...,open
109053,Amtali Upazila is the terminus of,R881,Amtali Upazila,reverse-http://yago-knowledge.org/resource/ter...,open


In [10]:
dataset = Dataset.from_pandas(df_all) #.select(range(256))

In [11]:
results, metrics = evaluate_model(model, tokenizer, dataset, batch_sz=360)

  0%|          | 0/303 [00:00<?, ?it/s]/cluster/home/kevidu/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/cluster/home/kevidu/venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
  0%|          | 1/303 [00:08<43:21,  8.61s/it]

Average accuracy at batch 0: 0.008333333333333333 (3/360).


  1%|          | 2/303 [00:13<32:22,  6.45s/it]

Average accuracy at batch 1: 0.008333333333333333 (6/720).


  1%|          | 3/303 [00:17<27:23,  5.48s/it]

...

100%|█████████▉| 302/303 [21:37<00:03,  4.00s/it]

Average accuracy at batch 301: 0.028182487122884473 (3064/108720).


100%|██████████| 303/303 [21:41<00:00,  4.30s/it]

Average accuracy at batch 302: 0.028095914905323004 (3064/109055).
109055



Map: 100%|██████████| 109055/109055 [00:00<00:00, 1595530.32 examples/s]


In [12]:
results_df = results.to_pandas()

In [13]:
correct_results_df = results_df[results_df["is_correct"] == True]
correct_results_df

,text,labels,entity,query_id,query_type,predictions,is_correct
42,'Eason Chan filmography' is about,Eason Chan,Eason Chan filmography,http://schema.org/about,open,"Eason Chan, a Hong Kong singer, songwriter, an...",True
105,'Norma Talmadge filmography' is about,Norma Talmadge,Norma Talmadge filmography,http://schema.org/about,open,"Norma Talmadge, an American actress who was ac...",True
196,'Faye Dunaway filmography' is about,Faye Dunaway,Faye Dunaway filmography,http://schema.org/about,open,"Faye Dunaway, an American actress who was born...",True
481,'Paul Anka filmography' is about,Paul Anka,Paul Anka filmography,http://schema.org/about,open,"Paul Anka, a Canadian singer, songwriter, and ...",True
585,'Donald Duck filmography' is about,Donald Duck,Donald Duck filmography,http://schema.org/about,open,"Donald Duck's appearances in various films, TV...",True
...,...,...,...,...,...,...,...
107070,Nykøbing-Rørvig Municipality was replaced by,Odsherred Municipality,Nykøbing-Rørvig Municipality,reverse-http://yago-knowledge.org/resource/rep...,open,"Odsherred Municipality on January 1, 2007. Unt...",True
107114,Öland County was replaced by,Kalmar County,Öland County,reverse-http://yago-knowledge.org/resource/rep...,open,Kalmar County in 1998. Unterscheidung between ...,True
107256,Newtownabbey Borough Council was replaced by,Antrim and Newtownabbey Borough Council,Newtownabbey Borough Council,reverse-http://yago-knowledge.org/resource/rep...,open,Antrim and Newtownabbey Borough Council in 201...,True
107268,Ceredigion and Pembroke North was replaced by,Ceredigion,Ceredigion and Pembroke North,reverse-http://yago-knowledge.org/resource/rep...,open,Ceredigion and Pembrokeshire in 1996. Untersch...,True


In [14]:
correct_results_df["query_id"].value_counts()

query_id
http://yago-knowledge.org/resource/capital               457
http://schema.org/numberOfSeasons                        322
http://schema.org/officialLanguage                       202
http://schema.org/manufacturer                           200
http://schema.org/nationality                            172
                                                        ... 
http://schema.org/numberOfEmployees                        2
http://schema.org/owns                                     1
reverse-http://yago-knowledge.org/resource/parentBody      1
reverse-http://schema.org/sponsor                          1
reverse-http://yago-knowledge.org/resource/studentOf       1
Name: count, Length: 66, dtype: int64

In [15]:
print("Num query ids:", len(correct_results_df["query_id"].unique()))
for qid in correct_results_df["query_id"].unique():
    print(qid)
    print(correct_results_df[correct_results_df["query_id"] == qid].iloc[0])
    print("\n\n")

Num query ids: 66
http://schema.org/about
text                           'Eason Chan filmography' is about
labels                                                Eason Chan
entity                                    Eason Chan filmography
query_id                                 http://schema.org/about
query_type                                                  open
predictions    Eason Chan, a Hong Kong singer, songwriter, an...
is_correct                                                  True
Name: 42, dtype: object



http://schema.org/address
text                          Dallas Museum of Art is located at
labels                                 1717 North Harwood Street
entity                                      Dallas Museum of Art
query_id                               http://schema.org/address
query_type                                                  open
predictions    1717 North Harwood Street, Dallas, TX 75201. e...
is_correct                                                  

In [16]:
correct_results_df

,text,labels,entity,query_id,query_type,predictions,is_correct
42,'Eason Chan filmography' is about,Eason Chan,Eason Chan filmography,http://schema.org/about,open,"Eason Chan, a Hong Kong singer, songwriter, an...",True
105,'Norma Talmadge filmography' is about,Norma Talmadge,Norma Talmadge filmography,http://schema.org/about,open,"Norma Talmadge, an American actress who was ac...",True
196,'Faye Dunaway filmography' is about,Faye Dunaway,Faye Dunaway filmography,http://schema.org/about,open,"Faye Dunaway, an American actress who was born...",True
481,'Paul Anka filmography' is about,Paul Anka,Paul Anka filmography,http://schema.org/about,open,"Paul Anka, a Canadian singer, songwriter, and ...",True
585,'Donald Duck filmography' is about,Donald Duck,Donald Duck filmography,http://schema.org/about,open,"Donald Duck's appearances in various films, TV...",True
...,...,...,...,...,...,...,...
107070,Nykøbing-Rørvig Municipality was replaced by,Odsherred Municipality,Nykøbing-Rørvig Municipality,reverse-http://yago-knowledge.org/resource/rep...,open,"Odsherred Municipality on January 1, 2007. Unt...",True
107114,Öland County was replaced by,Kalmar County,Öland County,reverse-http://yago-knowledge.org/resource/rep...,open,Kalmar County in 1998. Unterscheidung between ...,True
107256,Newtownabbey Borough Council was replaced by,Antrim and Newtownabbey Borough Council,Newtownabbey Borough Council,reverse-http://yago-knowledge.org/resource/rep...,open,Antrim and Newtownabbey Borough Council in 201...,True
107268,Ceredigion and Pembroke North was replaced by,Ceredigion,Ceredigion and Pembroke North,reverse-http://yago-knowledge.org/resource/rep...,open,Ceredigion and Pembrokeshire in 1996. Untersch...,True


In [19]:
yago_qec = load_dataset_from_path(RAW_DATA_PATH)

In [36]:
qid_to_entity_df = correct_results_df.groupby("query_id")["entity"].apply(list).reset_index()
qid_to_valid_entities = {row["query_id"]: set(row["entity"]) for _, row in qid_to_entity_df.iterrows()}

In [29]:
{k: len(v) for k, v in yago_qec['http://schema.org/about'].items()}

{'answer_types': 6,
 'answer_uris': 954,
 'answers': 954,
 'context_templates': 6,
 'entities': 954,
 'entity_namesake_to_degree': 954,
 'entity_namesake_to_num_uris': 954,
 'entity_types': 3,
 'entity_uri_to_degree': 954,
 'entity_uri_to_predicate_degree': 954,
 'entity_uris': 954,
 'gpt_fake_entities': 954,
 'query_forms': 2}

In [ ]:
{
    qid: {
        qec
    } for qid, qec in yago_qec.items()
}

In [39]:
list(yago_qec.keys())
eligible_yago_qec = dict()
for qid, qec in yago_qec.items():
    if qid in qid_to_valid_entities:
        eligible_inds = [i for i, e in enumerate(qec["entities"]) if e in qid_to_valid_entities[qid]]
        eligible_qec = {
            k: [x for i, x in enumerate(qec[k]) if i in eligible_inds] for k, v in qec.items() if len(v) == len(qec["entities"]) # Filter out the eligible indices for all keys parallel with entities (e.g. answers, etc.)
        }
        eligible_qec["context_templates"] = qec["context_templates"]
        eligible_qec["query_forms"] = qec["query_forms"]
        eligible_yago_qec[qid] = eligible_qec

In [41]:
list(eligible_yago_qec.keys())
eligible_yago_qec['http://schema.org/about']

{'answer_uris': ['http://yago-knowledge.org/resource/Morgan_Freeman',
  'http://yago-knowledge.org/resource/Gardening_generic_instance',
  'http://yago-knowledge.org/resource/Suriya',
  'http://yago-knowledge.org/resource/Nathan_Lane',
  'http://yago-knowledge.org/resource/Shilpa_Shetty',
  'http://yago-knowledge.org/resource/Norma_Talmadge',
  'http://yago-knowledge.org/resource/Faye_Dunaway',
  'http://yago-knowledge.org/resource/Donald_Duck',
  'http://yago-knowledge.org/resource/Eason_Chan',
  'http://yago-knowledge.org/resource/Paul_Anka'],
 'answers': ['Morgan Freeman',
  'gardening',
  'Suriya',
  'Nathan Lane',
  'Shilpa Shetty',
  'Norma Talmadge',
  'Faye Dunaway',
  'Donald Duck',
  'Eason Chan',
  'Paul Anka'],
 'entities': ['Morgan Freeman filmography',
  'Gardening for the Million',
  'Suriya filmography',
  'Nathan Lane on screen and stage',
  'Shilpa Shetty filmography',
  'Norma Talmadge filmography',
  'Faye Dunaway filmography',
  'Donald Duck filmography',
  'Eason 

In [42]:
correct_results_df[correct_results_df["query_id"] == 'http://schema.org/about']

,text,labels,entity,query_id,query_type,predictions,is_correct
42,'Eason Chan filmography' is about,Eason Chan,Eason Chan filmography,http://schema.org/about,open,"Eason Chan, a Hong Kong singer, songwriter, an...",True
105,'Norma Talmadge filmography' is about,Norma Talmadge,Norma Talmadge filmography,http://schema.org/about,open,"Norma Talmadge, an American actress who was ac...",True
196,'Faye Dunaway filmography' is about,Faye Dunaway,Faye Dunaway filmography,http://schema.org/about,open,"Faye Dunaway, an American actress who was born...",True
481,'Paul Anka filmography' is about,Paul Anka,Paul Anka filmography,http://schema.org/about,open,"Paul Anka, a Canadian singer, songwriter, and ...",True
585,'Donald Duck filmography' is about,Donald Duck,Donald Duck filmography,http://schema.org/about,open,"Donald Duck's appearances in various films, TV...",True
686,'Gardening for the Million' is about,gardening,Gardening for the Million,http://schema.org/about,open,"gardening for the masses, not just the wealthy...",True
762,'Morgan Freeman filmography' is about,Morgan Freeman,Morgan Freeman filmography,http://schema.org/about,open,"Morgan Freeman, an American actor, producer, a...",True
822,'Shilpa Shetty filmography' is about,Shilpa Shetty,Shilpa Shetty filmography,http://schema.org/about,open,"Shilpa Shetty's filmography, including her mov...",True
946,'Nathan Lane on screen and stage' is about,Nathan Lane,Nathan Lane on screen and stage,http://schema.org/about,open,"Nathan Lane's career as an actor, including hi...",True
948,'Suriya filmography' is about,Suriya,Suriya filmography,http://schema.org/about,open,"Suriya's filmography, which includes his movie...",True


In [45]:
import json
with open(FILTERED_RAW_DATA_PATH, "w", encoding="utf-8") as fp:
    json.dump(eligible_yago_qec, fp, ensure_ascii=False, indent=4, sort_keys=True)